In [1]:
!nvidia-smi -L
import torch, torchvision
print("Torch:", torch.__version__, "CUDA:", torch.version.cuda, "Available:", torch.cuda.is_available())

!pip install -q torch-fidelity==0.3.0

GPU 0: NVIDIA GeForce RTX 4090 (UUID: GPU-21484c64-3745-f39d-e1d4-e2e5fb24c475)
Torch: 2.1.2 CUDA: 12.1 Available: True


In [2]:
from pathlib import Path
import os, random, numpy as np
from PIL import Image

def seed_all(s=42):
    random.seed(s); np.random.seed(s); torch.manual_seed(s); torch.cuda.manual_seed_all(s)
seed_all()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

DATA_TRAIN = Path("data/raw/train/train")
DATA_VAL   = Path("data/raw/test/test")

OUTDIR = Path("./outputs_spade_finetune"); OUTDIR.mkdir(exist_ok=True)
CKPT_DIR   = OUTDIR/"checkpoints"; CKPT_DIR.mkdir(exist_ok=True)
SAMPLE_DIR = OUTDIR/"samples"; SAMPLE_DIR.mkdir(exist_ok=True)
PRED_DIR   = OUTDIR/"pred"; PRED_DIR.mkdir(exist_ok=True)
GT_DIR     = OUTDIR/"gt_photos"; GT_DIR.mkdir(exist_ok=True)

In [3]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch

class RandomJitter256:
    def __init__(self, jitter=True): self.jitter=jitter
    def __call__(self, img):
        img = img.resize((512,256), Image.BICUBIC)
        if self.jitter and random.random()<0.5: img = img.transpose(Image.FLIP_LEFT_RIGHT)
        return img

to_tensor_norm = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3,[0.5]*3)
])

class SideBySideDataset(Dataset):
    def __init__(self, root, train=True):
        self.root = Path(root); self.files = sorted(self.root.glob("*.jpg"))
        self.rj = RandomJitter256(jitter=train)
    def __len__(self): return len(self.files)
    def __getitem__(self, idx):
        img = Image.open(self.files[idx]).convert("RGB")
        img = self.rj(img)
        w,h = img.size; w2=w//2
        left = img.crop((0,0,w2,h)).resize((256,256),Image.BICUBIC)
        right= img.crop((w2,0,w,h)).resize((256,256),Image.BICUBIC)
        return to_tensor_norm(right), to_tensor_norm(left), self.files[idx].name

BATCH=8
train_loader = DataLoader(SideBySideDataset(DATA_TRAIN,True), batch_size=BATCH, shuffle=True, num_workers=4, pin_memory=True, drop_last=True)
val_loader   = DataLoader(SideBySideDataset(DATA_VAL,False),  batch_size=8,   shuffle=False, num_workers=4, pin_memory=True)

cond,target,_=next(iter(train_loader))
print("Shapes -> cond:",cond.shape," target:",target.shape)
assert cond.shape[-1]==256 and cond.shape[-2]==256

Shapes -> cond: torch.Size([8, 3, 256, 256])  target: torch.Size([8, 3, 256, 256])


In [9]:
import torch.nn as nn
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#  SPADE
class SPADE(nn.Module):
    def __init__(self, ch, seg_nc=3, hidden=128):
        super().__init__()
        self.norm = nn.InstanceNorm2d(ch, affine=False)
        self.mlp  = nn.Sequential(
            nn.Conv2d(seg_nc, hidden, 3, padding=1),
            nn.ReLU(True),
            nn.Conv2d(hidden, ch*2, 3, padding=1),
        )
    def forward(self, x, seg):
        seg = F.interpolate(seg, size=x.shape[-2:], mode="nearest")
        h   = self.norm(x)
        gamma, beta = torch.chunk(self.mlp(seg), 2, dim=1)
        return h*(1+gamma) + beta

class SPADEConv(nn.Module):
    def __init__(self, in_c, out_c, seg_nc=3, k=4, s=2, p=1, dropout=False):
        super().__init__()
        self.conv  = nn.Conv2d(in_c, out_c, k, s, p, bias=False)
        self.spade = SPADE(out_c, seg_nc)
        self.act   = nn.LeakyReLU(0.2, True)
        self.dp    = nn.Dropout(0.5) if dropout else nn.Identity()
    def forward(self, x, seg):
        return self.dp(self.act(self.spade(self.conv(x), seg)))

class SPADEResUp(nn.Module):
    def __init__(self, in_c, out_c, seg_nc=3, dropout=False):
        super().__init__()
        self.up   = nn.Upsample(scale_factor=2, mode="nearest")  # <- important
        self.conv = SPADEConv(in_c, out_c, seg_nc, k=3, s=1, p=1, dropout=dropout)
    def forward(self, x, seg):
        return self.conv(self.up(x), seg)

class UNetGenerator_SPADE(nn.Module):
    def __init__(self, in_ch=3, out_ch=3, seg_nc=3):
        super().__init__()
        #  Encoder
        self.d1 = SPADEConv(in_ch,   64,  seg_nc)  # 256→128
        self.d2 = SPADEConv(64,     128, seg_nc)   # 128→64
        self.d3 = SPADEConv(128,    256, seg_nc)   # 64→32
        self.d4 = SPADEConv(256,    512, seg_nc)   # 32→16
        self.d5 = SPADEConv(512,    512, seg_nc)   # 16→8
        self.d6 = SPADEConv(512,    512, seg_nc)   # 8→4
        self.d7 = SPADEConv(512,    512, seg_nc)   # 4→2  ← bottleneck

        # Decoder
        # Each upsample doubles spatial size and halves channels, then concatenates skip features
        self.u1 = SPADEResUp(512, 512, seg_nc, dropout=True)   # 2→4,  concat d6 (512) => 1024
        self.u2 = SPADEResUp(1024,512, seg_nc, dropout=True)   # 4→8,  concat d5 => 1024
        self.u3 = SPADEResUp(1024,512, seg_nc, dropout=True)   # 8→16, concat d4 => 1024
        self.u4 = SPADEResUp(1024,512, seg_nc)                 # 16→32, concat d3 => 768
        self.u5 = SPADEResUp(768,256, seg_nc)                  # 32→64, concat d2 => 384
        self.u6 = SPADEResUp(384,128, seg_nc)                  # 64→128, concat d1 => 192
        self.u7 = SPADEResUp(192,64,  seg_nc)                  # 128→256

        self.outc = nn.Conv2d(64, out_ch, 3, 1, 1)
        self.tanh = nn.Tanh()

    def forward(self, seg):
        #  Encoder
        d1 = self.d1(seg, seg)   # 128
        d2 = self.d2(d1, seg)    # 64
        d3 = self.d3(d2, seg)    # 32
        d4 = self.d4(d3, seg)    # 16
        d5 = self.d5(d4, seg)    # 8
        d6 = self.d6(d5, seg)    # 4
        d7 = self.d7(d6, seg)    # 2

        # Decoder
        u1 = self.u1(d7, seg);  u1 = torch.cat([u1, d6], 1)
        u2 = self.u2(u1, seg);  u2 = torch.cat([u2, d5], 1)
        u3 = self.u3(u2, seg);  u3 = torch.cat([u3, d4], 1)
        u4 = self.u4(u3, seg);  u4 = torch.cat([u4, d3], 1)
        u5 = self.u5(u4, seg);  u5 = torch.cat([u5, d2], 1)
        u6 = self.u6(u5, seg);  u6 = torch.cat([u6, d1], 1)
        u7 = self.u7(u6, seg)
        return self.tanh(self.outc(u7))

#  Discriminator (SN PatchGAN)
def snconv(ic, oc, k, s, p, bias=True):
    return nn.utils.spectral_norm(nn.Conv2d(ic, oc, k, s, p, bias=bias))

class PatchDiscriminatorSN(nn.Module):
    def __init__(self, in_ch=6):
        super().__init__()
        self.blocks = nn.ModuleList([
            nn.Sequential(snconv(in_ch,   64, 4,2,1),              nn.LeakyReLU(0.2,True)),
            nn.Sequential(snconv(64,     128, 4,2,1, False), nn.InstanceNorm2d(128), nn.LeakyReLU(0.2,True)),
            nn.Sequential(snconv(128,    256, 4,2,1, False), nn.InstanceNorm2d(256), nn.LeakyReLU(0.2,True)),
            nn.Sequential(snconv(256,    512, 4,1,1, False), nn.InstanceNorm2d(512), nn.LeakyReLU(0.2,True)),  # stop downsampling here
        ])
        self.head = snconv(512, 1, 4,1,1)

    def forward(self, x):
        feats = []
        h = x
        for b in self.blocks:
            h = b(h); feats.append(h)
        return self.head(h), feats

# instantiate
G = UNetGenerator_SPADE().to(device)
D = PatchDiscriminatorSN().to(device)
print(f"Params G: {sum(p.numel() for p in G.parameters())/1e6:.2f}M, D: {sum(p.numel() for p in D.parameters())/1e6:.2f}M")

Params G: 45.74M, D: 2.77M


In [14]:
from torchvision.models import vgg16, VGG16_Weights
import torch.nn.functional as F
from torch_fidelity import calculate_metrics

def d_hinge(r,f): return (F.relu(1.-r).mean()+F.relu(1.+f).mean())
def g_hinge(f): return -f.mean()
l1 = nn.L1Loss()

vgg=vgg16(weights=VGG16_Weights.IMAGENET1K_FEATURES).features[:16].eval().to(device)
for p in vgg.parameters(): p.requires_grad=False
def perceptual(x,y): return l1(vgg(x),vgg(y))
def denorm(x): return (x*0.5+0.5).clamp(0,1)

# fine-tuned weights
LAMBDA_L1, LAMBDA_FM, LAMBDA_VGG = 35.0, 5.0, 5.0
LR_G, LR_D = 2e-4, 2.5e-4
EPOCHS, SAVE_EVERY, EVAL_EVERY = 250, 50, 10

opt_G=torch.optim.Adam(G.parameters(),lr=LR_G,betas=(0.5,0.999))
opt_D=torch.optim.Adam(D.parameters(),lr=LR_D,betas=(0.5,0.999))
scheduler_G=torch.optim.lr_scheduler.StepLR(opt_G,step_size=400,gamma=0.5)
scheduler_D=torch.optim.lr_scheduler.StepLR(opt_D,step_size=400,gamma=0.5)

def compute_fid(gt_dir, pred_dir, cuda=True):
    m = calculate_metrics(
        input1=str(gt_dir),
        input2=str(pred_dir),
        cuda=cuda and torch.cuda.is_available(),
        isc=False, fid=True, kid=False, verbose=False
    )
    return float(m['frechet_inception_distance'])

In [15]:
from torchvision.utils import make_grid
from torchvision import transforms
from tqdm.auto import tqdm

def train_spade(train_loader,val_loader):
    best_fid=float("inf")
    for epoch in range(1,EPOCHS+1):
        G.train(); D.train()
        epG=epD=epL1=0.
        pbar=tqdm(train_loader,desc=f"Epoch {epoch}/{EPOCHS}")
        for cond,target,_ in pbar:
            cond,target=cond.to(device),target.to(device)

            # --- tiny contrast jitter ---
            if random.random()<0.25:
                cond=cond*(0.9+0.2*torch.rand(1,device=cond.device))

            # ----- D -----
            with torch.no_grad(): 
                fake=G(cond)
            rp=torch.cat([cond,target],1); 
            fp=torch.cat([cond,fake],1)
            
            D.zero_grad(set_to_none=True)
            rlog,_=D(rp);
            flog,_=D(fp)
            loss_D=d_hinge(rlog,flog)
            loss_D.backward(); 
            opt_D.step()

            # ----- G -----
            G.zero_grad(set_to_none=True)
            gen=G(cond); 
            gp=torch.cat([cond,gen],1)
            glog,gf=D(gp)
            loss_G_adv=g_hinge(glog)
            loss_G_L1=l1(gen,target)*LAMBDA_L1
            
            with torch.no_grad():
                _,rf=D(rp)
            loss_G_FM=sum(l1(g,r) for g,r in zip(gf,rf))*LAMBDA_FM
            loss_G_VGG=perceptual(denorm(gen),denorm(target))*LAMBDA_VGG
            
            loss_G=loss_G_adv+loss_G_L1+loss_G_FM+loss_G_VGG
            loss_G.backward(); 
            opt_G.step()

            epG+=loss_G.item(); 
            epD+=loss_D.item(); 
            epL1+=loss_G_L1.item()
            pbar.set_postfix(D=f"{loss_D.item():.3f}",G=f"{loss_G.item():.2f}",L1=f"{loss_G_L1.item():.2f}")

        scheduler_G.step(); scheduler_D.step()
        avgG,avgD,avgL1=epG/len(train_loader),epD/len(train_loader),epL1/len(train_loader)
        print(f"Epoch {epoch}: G={avgG:.3f} D={avgD:.3f} L1={avgL1:.3f}")

        # save sample grid every 10
        if epoch%10==0:
            G.eval()
            with torch.no_grad():
                c,t,_=next(iter(val_loader)); c=c.to(device)
                g=G(c)
                grid=make_grid(torch.cat([denorm(c.cpu()),denorm(g.cpu()),denorm(t)],0),nrow=c.size(0))
                img=transforms.ToPILImage()(grid)
                img.save(SAMPLE_DIR/f"epoch_{epoch:03d}.jpg",quality=95)

        # checkpoints
        if epoch%SAVE_EVERY==0:
            ckpt=CKPT_DIR/f"epoch_{epoch}.pt"
            torch.save({"G":G.state_dict(),"D":D.state_dict(),"epoch":epoch},ckpt)
            print("Checkpoint saved:",ckpt)

        # generate preds + FID 
        if epoch%EVAL_EVERY==0:
            for p in PRED_DIR.glob("*.jpg"): p.unlink()
            G.eval()
            with torch.no_grad():
                for c,_,names in tqdm(val_loader,desc="Predict val"):
                    c=c.to(device)
                    g=denorm(G(c))
                    for i,n in enumerate(names):
                        transforms.ToPILImage()(g[i].cpu()).save(PRED_DIR/n,quality=95)
            fid=compute_fid(GT_DIR,PRED_DIR)
            print(f"→ FID @ epoch {epoch}: {fid:.2f}")
            if fid<best_fid:
                best_fid=fid
                torch.save({"G":G.state_dict(),"D":D.state_dict(),
                            "epoch":epoch,"fid":fid},CKPT_DIR/"best_model.pt")
                print(f"New best FID {fid:.2f} @ epoch {epoch}")

In [12]:
from tqdm.auto import tqdm
for f in tqdm(sorted(DATA_VAL.glob("*.jpg")),desc="Extract GT photos"):
    im=Image.open(f).convert("RGB").resize((512,256),Image.BICUBIC)
    w,h=im.size; photo=im.crop((0,0,w//2,h)).resize((256,256),Image.BICUBIC)
    photo.save(GT_DIR/f.name,quality=95)
print("GT count:",len(list(GT_DIR.glob("*.jpg"))))

Extract GT photos: 100%|██████████| 500/500 [00:09<00:00, 55.16it/s]

GT count: 500


In [16]:
# load from last or start fresh
ckpt = CKPT_DIR/"epoch_200.pt"
if ckpt.exists():
    c=torch.load(ckpt,map_location=device)
    G.load_state_dict(c["G"]); D.load_state_dict(c["D"])
    print("Resumed from",ckpt)
else:
    print("Training from scratch")

train_spade(train_loader,val_loader)

Training from scratch


Epoch 1/250: 100%|██████████| 371/371 [00:50<00:00,  7.28it/s, D=0.675, G=16.90, L1=4.93]


Epoch 1: G=18.474 D=1.073 L1=5.213


Epoch 2/250: 100%|██████████| 371/371 [00:50<00:00,  7.31it/s, D=1.142, G=18.22, L1=5.16]


Epoch 2: G=18.343 D=1.077 L1=5.164


...

Epoch 250/250: 100%|██████████| 371/371 [00:50<00:00,  7.35it/s, D=0.006, G=14.77, L1=2.50]

Epoch 250: G=14.340 D=0.033 L1=2.525


Checkpoint saved: outputs_spade_finetune/checkpoints/epoch_250.pt


Predict val: 100%|██████████| 63/63 [00:05<00:00, 12.52it/s]


→ FID @ epoch 250: 107.63


In [18]:
best=torch.load(CKPT_DIR/"best_model.pt",map_location=device)
G.load_state_dict(best["G"])
print(f"Loaded best model epoch {best['epoch']} FID {best['fid']:.2f}")

# regenerate predictions + compute FID
for p in PRED_DIR.glob("*.jpg"): p.unlink()
G.eval()
with torch.no_grad():
    for c,_,names in tqdm(val_loader,desc="Predict final"):
        c=c.to(device)
        g=denorm(G(c))
        for i,n in enumerate(names):
            transforms.ToPILImage()(g[i].cpu()).save(PRED_DIR/n,quality=95)
fid=compute_fid(GT_DIR,PRED_DIR)
print("Final FID:",round(fid,2))

Loaded best model epoch 70 FID 85.45


Predict final: 100%|██████████| 63/63 [00:04<00:00, 12.61it/s]


Final FID: 85.45
